## AI-SQL  For Healthcare

Snowflake's AI-SQL capabilities listed below simplifies usage of AI on multimodal Healthcare data , for SQL Savvy users
1) FILE DATA TYPE
2) AI_FILTER
3) AI_TRANSCRIBE
4) AI_CLASSIFY
5) AI_AGG

In [ ]:
import streamlit as st
import pandas as pd
import snowbooks_extras

## Leverage AISQL to process images for Healthcare

Pt1: Load Medical Images directly from stage using newly introduced FILE Data Type. Leverage 

Usefull for multiple usecases for.e.g Identify medical images to the appropriata Data Product pipelines. E.g 'Retinal Scan' images into a Vision Health Data Product.

In [ ]:
-- uploaded medical images(jpegs) into stage @AI_SQL_TEAM_DB.HCLS_SE_SAMPLE_DATA.HCLS_SE_SAMPLE_STAGE/MEDICAL_I
-- (note: please create stage as client side encryption)

create or replace  table medical_images as
select TO_FILE(file_url) img_file, * 
FROM directory(@AI_SQL_DEMO_DB.HEALTHCARE_DEMO.MEDICAL_IMAGES);
-- WHERE RELATIVE_PATH LIKE 'MEDICAL_IMAGES/%';

select * from medical_images;



In [ ]:
SELECT 
IMG_FILE,
AI_COMPLETE('claude-3-7-sonnet',
PROMPT('Classify the input image {0} in no more than 2 words', img_file)) AS image_classification
FROM medical_images;

In [ ]:
SELECT 
IMG_FILE
FROM medical_images
WHERE AI_FILTER(PROMPT('This medical image {0} is a Chest X-ray', img_file)) ;

In [ ]:
SELECT 
    img_file,
    AI_CLASSIFY(PROMPT('please help me classify this medical scan image {0}', img_file),
        ['X-ray', 'CT', 'MRI', 'Ultrasound','PET','Other']) as classification
from medical_images;


## Working with PDFs via AISQL Functions

Use AI_FILTER to:
 1) Filter out Medical Transcripts that are related to a specific ICD10 value.


In [ ]:
CREATE OR REPLACE TABLE POP_HEALTH_MEDICAL_TRANSCRIPTS_RAW AS SELECT
    relative_path as TRANSCRIPT_NAME, 
    GET_PRESIGNED_URL(@AI_SQL_DEMO_DB.HEALTHCARE_DEMO.Medical_Transcripts, relative_path) as scoped_file_url, 
    TO_VARIANT(SNOWFLAKE.CORTEX.PARSE_DOCUMENT(@AI_SQL_DEMO_DB.HEALTHCARE_DEMO.Medical_Transcripts, relative_path , {'mode': 'ocr'})) as raw_text_dict,
    raw_text_dict:content as FULL_TEXT
FROM DIRECTORY(@AI_SQL_DEMO_DB.HEALTHCARE_DEMO.Medical_Transcripts);


In [ ]:
--MEDICAL TRANSCRIPTS information is preloaded into a Snowflake STRING column using Parse_Document function.
SELECT TRANSCRIPT_NAME, FULL_TEXT FROM POP_HEALTH_MEDICAL_TRANSCRIPTS_RAW

In [ ]:
--Filtering for medical transcripts which refer to a motor accident

SELECT TRANSCRIPT_NAME, FULL_TEXT FROM POP_HEALTH_MEDICAL_TRANSCRIPTS_RAW
WHERE AI_FILTER('This medical transcript has references to a motor vehicle accident faced by the member' ||  FULL_TEXT)

## Leverage AISQL to process audios for Healthcare

Load Audio files directly from stage using newly introduced FILE Data Type

Leverage AI_TRANSCRIBE function 

In [ ]:
SELECT  
        A.RELATIVE_PATH ,
        A.CALL_TRANSCRIPT,
        AI_CLASSIFY(CALL_TRANSCRIPT, ['claims related', 'coverage related', 'billing related','other']):labels[0]::text AI_CALL_REASON
FROM 
(SELECT RELATIVE_PATH, AI_TRANSCRIBE(audio_file):text::STRING CALL_TRANSCRIPT FROM
    (SELECT 
        relative_path RELATIVE_PATH,
        TO_FILE('@AI_SQL_DEMO_DB.HEALTHCARE_DEMO.Call_Recordings', relative_path) AS audio_file 
    FROM DIRECTORY(@AI_SQL_DEMO_DB.HEALTHCARE_DEMO.Call_Recordings)
    -- WHERE RELATIVE_PATH LIKE  '%CALL_RECORDINGS%'
    LIMIT 5
    )
) A
    

## Get aggregated insights directly via SQL

Use AI_AGG to:

Generate Actionable health intervention insights aggregated at population level across multiple medical transcripts

In [ ]:
SELECT AI_AGG(FULL_TEXT, '

You are an AI agent helping the physican assigned for a population of patients.And these are the medical transcripts associated to those patients.
Evaluate these medical transcripts, and recommend the following.

1) Key insights on the overall health of the undertaken population
2) What are the specific actions at a population level that they physician can take to improve the health
3) Who is the top patient that needs highest level of attention , and what are the proposed next steps in engaging with that patient?

') ACTIONABLE_INSIGHTS
FROM POP_HEALTH_MEDICAL_TRANSCRIPTS_RAW

In [ ]:
agg_patient_level_sdoh_insights_df = ai_agg_patient_level_sdoh_insights_1.to_df().to_pandas()
for insight in agg_patient_level_sdoh_insights_df['ACTIONABLE_INSIGHTS']:
    st.write(insight)
    # Or using markdown for better formatting potential
    # st.markdown(f"- {insight}")